In [1]:
import mysql.connector
from getpass import getpass
import hashlib
from tensorflow.keras.models import load_model
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from groq import Groq

In [2]:
# Load NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Load the trained model
model = load_model('emotion_classification_model.h5')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vedag\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vedag\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vedag\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
# Example training texts (used to fit the tokenizer)
texts = [
    "I feel really happy today!",
    "This is terrible...",
    "I love this!",
    "I'm so angry right now.",
    "This makes me scared.",
    "Wow, I'm surprised!"
]

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts)

# Preprocessing function
def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens if token.isalpha() and token not in stop_words]
    return ' '.join(tokens)

# Emotion prediction function
def predict_emotion(text, tokenizer, model):
    processed_text = preprocess_text(text)
    sequence = tokenizer.texts_to_sequences([processed_text])
    padded_sequence = pad_sequences(sequence, maxlen=40)
    prediction = model.predict(padded_sequence)
    emotion_index = np.argmax(prediction)
    emotions = ["joy", "sadness", "love", "anger", "fear", "surprise"]
    return emotions[emotion_index]


MySQL Integration

In [6]:
def get_db_connection():
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="root",
        database="chatbot_db"
    )
    return connection

def create_user(username, password):
    connection = get_db_connection()
    cursor = connection.cursor()
    hashed_password = hashlib.sha256(password.encode()).hexdigest()
    cursor.execute("INSERT INTO users (username, password) VALUES (%s, %s)", (username, hashed_password))
    connection.commit()
    cursor.close()
    connection.close()

def authenticate_user(username, password):
    connection = get_db_connection()
    cursor = connection.cursor()
    hashed_password = hashlib.sha256(password.encode()).hexdigest()
    cursor.execute("SELECT user_id FROM users WHERE username = %s AND password = %s", (username, hashed_password))
    result = cursor.fetchone()
    cursor.close()
    connection.close()
    if result:
        return result[0]  
    return None

def store_conversation(user_id, user_input, bot_response, emotion):
    connection = get_db_connection()
    cursor = connection.cursor()
    cursor.execute("INSERT INTO chat_logs (user_id, user_input, bot_response, emotion) VALUES (%s, %s, %s, %s)",
                   (user_id, user_input, bot_response, emotion))
    connection.commit()
    cursor.close()
    connection.close()

def get_conversation_history(user_id):
    connection = get_db_connection()
    cursor = connection.cursor()
    cursor.execute("SELECT user_input, bot_response FROM chat_logs WHERE user_id = %s ORDER BY id ASC", (user_id,))
    result = cursor.fetchall()
    cursor.close()
    connection.close()
    return result


Groq Integration

In [7]:
# Initialize Groq client
api_key = "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"
client = Groq(api_key=api_key)

# Generating response using Groq API
def generate_response(prompt, api_key):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-70b-8192"
    )
    return chat_completion.choices[0].message.content

# Create emotional response function
def create_emotional_response(user_input, api_key, user_id):
    detected_emotion = predict_emotion(user_input, tokenizer, model)
    
    # Fetch conversation history
    conversation_history = get_conversation_history(user_id)
    
    # Formulate the prompt with history
    prompt = ""
    for user_msg, bot_resp in conversation_history:
        prompt += f"User: {user_msg}\nBot: {bot_resp}\n"
    prompt += f"User: {user_input}\nBot:"
    
    bot_response = generate_response(prompt, api_key)
    store_conversation(user_id, user_input, bot_response, detected_emotion)
    return bot_response


In [ ]:
username = input("Enter username: ")
password = getpass("Enter password: ")
create_user(username, password)
print("User registered successfully")



In [8]:
username = input("Enter username: ")
password = getpass("Enter password: ")
user_id = authenticate_user(username, password)
if user_id:
    print("Login successful")
else:
    print("Invalid credentials")

if user_id:
    while True:
        user_input = input("Enter your message (or type 'exit' to end): ")
        if user_input.lower() == 'exit':
            print("Conversation ended.")
            break
        response = create_emotional_response(user_input, api_key, user_id)
        print("Bot response:", response)
else:
    print("Please login first")


Enter username: Ved
Enter password: ········
Login successful
Enter your message (or type 'exit' to end): I am feeling sad
1/1 [==============================] - 0s 497ms/step
Bot response: I'm so sorry to hear that you're feeling sad. It can be really tough to deal with difficult emotions. Would you like to talk about what's going on and how you're feeling? Sometimes sharing what's on your mind can help you feel a bit better. I'm here to listen and offer support if you need it.
Enter your message (or type 'exit' to end): I lost my job
1/1 [==============================] - 0s 19ms/step
Bot response: Oh no, I'm so sorry to hear that you lost your job. That can be a really tough and stressful experience. It's normal to feel sad, angry, or frustrated when something like that happens. It's okay to feel those emotions and take time to process them.

Can you tell me a bit more about what happened? Was it sudden or was it something you saw coming? How are you feeling about the whole situatio